In [1]:
from preprocessing.prep import prepare_paths

2024-06-26 10:57:05.827635: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 10:57:06.497650: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import pandas as pd


In [3]:

args = pd.Series({
    "root_dir": "/mnt/disks/data/",
    "dataset_path": "/mnt/disks/data/fma/fma_large", 
    "embeddings": "music_style",
    "sequence_size": 1280,
    "train_id": "hierarchical_tworoots_dev",
    'sample_size': 0.1
})



In [4]:
tracks_df, args = prepare_paths(args)

In [5]:
tracks_df

,track_genres_all,track_id,file_path,index,track_genre_top
44144,"[42, 236, 495, 15, 181]",66333,/mnt/disks/data/fma/fma_large/066/066333.mp3,45071,Electronic
32789,[5],50948,/mnt/disks/data/fma/fma_large/050/050948.mp3,33488,Classical
91549,"[41, 38, 247]",137485,/mnt/disks/data/fma/fma_large/137/137485.mp3,93722,Experimental
10312,"[1, 38]",17197,/mnt/disks/data/fma/fma_large/017/017197.mp3,10468,Experimental
42034,[12],63774,/mnt/disks/data/fma/fma_large/063/063774.mp3,42944,Rock
...,...,...,...,...,...
72571,"[32, 38]",112404,/mnt/disks/data/fma/fma_large/112/112404.mp3,74373,Experimental
17026,"[2, 172]",27975,/mnt/disks/data/fma/fma_large/027/027975.mp3,17357,International
29688,"[1, 38, 41]",46010,/mnt/disks/data/fma/fma_large/046/046010.mp3,30303,Experimental
18123,"[240, 297, 15]",29527,/mnt/disks/data/fma/fma_large/029/029527.mp3,18501,Electronic


In [7]:
# Inicializar e ajustar o MultiLabelBinarizer
#mlb = MultiLabelBinarizer()
#mlb.fit(tracks_df.track_genres_all.values.tolist())

In [8]:
tracks_df.track_genres_all.iloc[2]

[1, 38, 41]

In [9]:
#mlb.transform([tracks_df.track_genres_all.iloc[0]])

In [10]:
# Cria um dicionário que associa o ID de cada música aos IDs de seus gêneros musicais
#tracks_df = pd.read_csv(os.path.join(args.metadata_path,'tracks.csv'), header=[0,1], index_col=0)

In [11]:
tracks_df.track_genre_top.value_counts()

track_genre_top
Rock                   1424
Experimental           1078
Electronic              962
Hip-Hop                 351
Folk                    287
Pop                     249
Instrumental            202
Classical               137
International           130
Jazz                     72
Spoken                   40
Old-Time / Historic      40
Soul-RnB                 16
Country                  13
Blues                     9
Easy Listening            2
Name: count, dtype: int64

In [12]:
tracks_df = tracks_df[tracks_df['track_genre_top'].isin(['Rock','Electronic'])]

In [13]:
tracks_df.columns

Index(['track_genres_all', 'track_id', 'file_path', 'index',
       'track_genre_top'],
      dtype='object')

In [7]:
import pandas as pd
import numpy as np
import shutil
import json
import ast
import os
from tqdm.notebook import tqdm

from utils.dir import create_dir, __load_json__
from dataset.labels import __create_labels__, get_labels_name, split_label, get_all_structure, convert_label_to_string
from dataset.dataset import select_dataset, create_metadata, load_features
from dataset.dataset_torch import generate_torch_data

In [8]:
def remover_sublistas_redundantes(lista_de_listas):
    max_depth = max([len(value) for value in lista_de_listas])
    new_sublist = []
    for sublista in lista_de_listas:
        if len(sublista) == max_depth:
            new_sublist.append(sublista)

    return new_sublist

In [17]:
tracks_df, args = prepare_paths(args)
tracks_df = tracks_df[tracks_df['track_genre_top'].isin(['Rock','Electronic'])]

In [18]:
tracks_df

,track_genres_all,track_id,file_path,index,track_genre_top
47836,"[25, 12]",71651,/mnt/disks/data/fma/fma_large/071/071651.mp3,48853,Rock
16034,"[98, 36, 12, 113, 26]",26485,/mnt/disks/data/fma/fma_large/026/026485.mp3,16346,Rock
89435,"[495, 236, 15]",134360,/mnt/disks/data/fma/fma_large/134/134360.mp3,91521,Electronic
66685,"[89, 12, 25]",104677,/mnt/disks/data/fma/fma_large/104/104677.mp3,68360,Rock
65873,"[25, 12, 109]",102188,/mnt/disks/data/fma/fma_large/102/102188.mp3,67543,Rock
...,...,...,...,...,...
69077,"[286, 15]",107800,/mnt/disks/data/fma/fma_large/107/107800.mp3,70823,Electronic
1958,[12],3656,/mnt/disks/data/fma/fma_large/003/003656.mp3,1966,Rock
5378,[15],10260,/mnt/disks/data/fma/fma_large/010/010260.mp3,5450,Electronic
40384,"[58, 12]",61490,/mnt/disks/data/fma/fma_large/061/061490.mp3,41277,Rock


In [19]:
tracks_df, args = prepare_labels(tracks_df,args)

max depth: 4


/tmp/ipykernel_325810/2221236487.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df.loc[:, labels_name[idx]] = binarized_labels
/tmp/ipykernel_325810/2221236487.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df.loc[:, labels_name[idx]] = binarized_labels
/tmp/ipykernel_325810/2221236487.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [20]:
len(tracks_df.level2.iloc[0])

29

In [21]:
tracks_df.y_true

47836                [[12, 25]]
16034           [[12, 26, 113]]
89435    [[15, 495], [15, 236]]
66685            [[12, 25, 89]]
65873           [[12, 25, 109]]
                  ...          
69077               [[15, 286]]
1958                     [[12]]
5378                     [[15]]
40384                [[12, 58]]
96041    [[15, 495], [15, 286]]
Name: y_true, Length: 2330, dtype: object

In [22]:
tracks_df.y_true.values.tolist()

[[[12, 25]],
 [[12, 26, 113]],
 [[15, 495], [15, 236]],
 [[12, 25, 89]],
 [[12, 25, 109]],
 [[15]],
 [[15]],
 [[12, 25, 111]],
 [[15, 495], [15, 236]],
 [[12, 66]],
 [[12, 25, 111]],
 [[12, 25, 109]],
 [[15]],
 [[12, 85]],
 [[12, 25], [12, 85]],
 [[12, 26]],
 [[15, 236], [15, 181]],
 [[12, 66], [12, 85]],
 [[12, 70]],
 [[15]],
 [[15, 297, 240]],
 [[12]],
 [[12, 25, 111]],
 [[12, 25, 111]],
 [[12]],
 [[15, 297, 240]],
 [[12, 25], [12, 66]],
 [[12]],
 [[15]],
 [[12, 66], [12, 27], [12, 359]],
 [[12, 26]],
 [[12, 25, 89], [12, 25, 71]],
 [[15, 296]],
 [[15, 495]],
 [[12, 25, 111]],
 [[12]],
 [[12]],
 [[12, 58]],
 [[15, 495]],
 [[12, 25]],
 [[12, 25]],
 [[15, 181]],
 [[12, 27]],
 [[15, 296], [15, 42]],
 [[12, 66], [12, 359]],
 [[12]],
 [[12, 25, 71], [12, 45, 53]],
 [[12, 45, 53]],
 [[15]],
 [[15, 183], [15, 236]],
 [[15, 495], [15, 286]],
 [[15, 42], [15, 236]],
 [[12, 25, 109]],
 [[15, 297, 240]],
 [[12, 25]],
 [[12, 66]],
 [[12, 85]],
 [[15, 181]],
 [[15, 468]],
 [[12]],
 [[12, 25], [12

In [23]:
max_depth = tracks_df.y_true.apply(lambda x: max([len(value) for value in x]))
max_depth = int(max_depth.max())

In [24]:
max_depth

4

In [26]:
args

root_dir                                                   /mnt/disks/data/
dataset_path                                  /mnt/disks/data/fma/fma_large
embeddings                                                      music_style
sequence_size                                                          1280
train_id                                          hierarchical_tworoots_dev
sample_size                                                             0.1
job_path                  /mnt/disks/data/fma/trains/hierarchical_tworoo...
tfrecord_path             /mnt/disks/data/fma/trains/hierarchical_tworoo...
torch_path                /mnt/disks/data/fma/trains/hierarchical_tworoo...
models_path                                          /mnt/disks/data/models
metadata_path                              /mnt/disks/data/fma/fma_metadata
metadata_train_path       /mnt/disks/data/fma/trains/hierarchical_tworoo...
categories_labels_path    /mnt/disks/data/fma/trains/hierarchical_tworoo...
max_depth   

In [ ]:

def split_dataset(tracks_df,args):
    #### Split dataset

    df_train, df_test, df_val = select_dataset(tracks_df, args)

    args['val_path'] = os.path.join(args.tfrecord_path, 'val')
    args['test_path'] = os.path.join(args.tfrecord_path, 'test')
    args['train_path'] = os.path.join(args.tfrecord_path, 'train')

    args['val_torch_path'] = os.path.join(args.torch_path, 'val.pth')
    args['test_torch_path'] = os.path.join(args.torch_path, 'test.pth')
    args['train_torch_path'] = os.path.join(args.torch_path, 'train.pth')

    args['train_csv'] = os.path.join(args.job_path, "train.csv")
    args['test_csv'] = os.path.join(args.job_path, "test.csv")
    args['val_csv'] = os.path.join(args.job_path, "val.csv")

    df_features = load_features(args.dataset_path, dataset=args.embeddings)

    df_features.dropna(inplace=True)

    df_val_features = df_val.merge(df_features, on='track_id')
    df_test_features = df_test.merge(df_features, on='track_id')
    df_train_features = df_train.merge(df_features, on='track_id')


    print(df_train_features.columns)
    print(type(df_train_features.y_true.iloc[0]))

    # df_train_features.to_csv(args['train_csv'], index=False)
    # df_test_features.to_csv(args['test_csv'], index=False)
    # df_val_features.to_csv(args['val_csv'], index=False)

    generate_tf_record(df_val_features, args, tf_path=args['val_path'])
    generate_tf_record(df_test_features, args, tf_path=args['test_path'])
    generate_tf_record(df_train_features, args, tf_path=args['train_path'])

    # generate_torch_data(df_val_features, args, save_path=args['val_torch_path'], batch_size=1024 * 50, shuffle=True)
    # generate_torch_data(df_test_features, args, save_path=args['test_torch_path'], batch_size=1024 * 50, shuffle=True)
    # generate_torch_data(df_train_features, args, save_path=args['train_torch_path'], batch_size=1024 * 50, shuffle=True)

    args['val_len'] = df_val.shape[0]
    args['test_len'] = df_test.shape[0]
    args['train_len'] = df_train.shape[0]

    # ## Create metadata file
    create_metadata(args)

In [27]:
split_dataset(tracks_df, args)

NameError: name 'split_dataset' is not defined

In [153]:
args.labels

{'label_1': {15: 0, 12: 1},
 'label_1_name': {15: 'Electronic', 12: 'Rock'},
 'label_1_inverse': [15, 12],
 'label_1_count': 2,
 'label_2': {85: 1,
  58: 2,
  468: 3,
  297: 4,
  45: 5,
  25: 6,
  27: 7,
  66: 8,
  495: 9,
  236: 10,
  286: 11,
  184: 12,
  183: 13,
  185: 14,
  26: 15,
  31: 16,
  42: 17,
  181: 18,
  359: 19,
  98: 20,
  314: 21,
  70: 22,
  182: 23,
  337: 24,
  695: 25,
  36: 26,
  88: 27,
  296: 28,
  440: 29},
 'label_2_name': {85: 'Rock>Garage',
  58: 'Rock>Psych-Rock',
  468: 'Electronic>Dubstep',
  297: 'Electronic>Chip Music',
  45: 'Rock>Loud-Rock',
  25: 'Rock>Punk',
  27: 'Rock>Lo-Fi',
  66: 'Rock>Indie-Rock',
  495: 'Electronic>Downtempo',
  236: 'Electronic>IDM',
  286: 'Electronic>Trip-Hop',
  184: 'Electronic>Minimal Electronic',
  183: 'Electronic>Glitch',
  185: 'Electronic>Breakcore - Hard',
  26: 'Rock>Post-Rock',
  31: 'Rock>Metal',
  42: 'Electronic>Ambient Electronic',
  181: 'Electronic>Techno',
  359: 'Rock>Shoegaze',
  98: 'Rock>Progressive',